# 5 train xencoder

In [1]:
# чистая модель, без прогрева, скорость обучения не задана, только 0 и 1, default_activation = sigmoid
# эпохи - от 5 раз, л, пока взять небольшое knn = 20-30
# кроссэнкодер путается, когда слишком много примеров и опирается на предыдущее обучение, нужно, чтобы он его забыл
# добавим descritption - иначе кросс энкодер путается

# возьмем новый биэнкодер и новые примеры 
# merged text - точно как в биэнкодере с пробелами
# давай все же оттренируем 5 эпох грязной модели ? и knn нам нужен выше 20
# сделаем полноценное k-fold кросс валидацию
# возможно одно из условий успеха - постоянная перегенерация примеров на дообученных моделях и снова перегенерация примеров

import sys
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
import string
from sentence_transformers import SentenceTransformer,InputExample,losses
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import util
import torch
from torch import nn
from torch.utils.data import DataLoader


print ('started')
#x_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2',num_labels=1, max_length=512, default_activation_function=nn.Sigmoid())
x_encoder = CrossEncoder('x-final-0311b',num_labels=1, max_length=128, default_activation_function=nn.Sigmoid())

BASE_DIR = 'C:/learning'              # временно, для отладки
num_epochs = 1
#warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data
warmup_steps = 0 
#lr = 2e-5



started


In [2]:
# k-fold validation

from sklearn.model_selection import KFold
from sentence_transformers.cross_encoder.evaluation import CERerankingEvaluator,CEBinaryAccuracyEvaluator,CECorrelationEvaluator

data = pd.read_csv('train_0311.csv')

kfold = KFold(3, shuffle = True, random_state = 42)
num_epochs = 1  
model_save_path = "x-0313"

for train_dataset_n, dev_dataset_n in kfold.split(data):

    train_dataset = data.iloc[train_dataset_n]
    dev_dataset = data.iloc[dev_dataset_n].sample(frac=0.2)
   
    #dev_examples=[]
    #dev_labels=[]
    dev_samples=[]
    for idx, row in tqdm(dev_dataset.iterrows(), total=dev_dataset.shape[0], position=0, leave=True):
        #dev_examples.append([row['topic'], row['content']])
        #dev_labels.append(row['label'])
        dev_samples.append(InputExample(texts=[row['topic'], row['content']], label=row['label']))
    
    train_examples = []
    for idx, row in tqdm(train_dataset.iterrows(), total=train_dataset.shape[0], position=0, leave=True):
        train_examples.append(InputExample(texts=[row['topic'], row['content']], label=row['label']))
        train_examples.append(InputExample(texts=[row['content'],row['topic']], label=row['label']))
    train_dataloader = DataLoader(train_examples, shuffle=False, batch_size=240)
    
    #dev_evaluator = CEBinaryAccuracyEvaluator(sentence_pairs= dev_examples, labels =dev_labels, name='train-eval')
    
    dev_evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='sts-dev')
    
    x_encoder.fit(train_dataloader=train_dataloader,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=dev_evaluator,
          evaluation_steps=30000,
          output_path=model_save_path,
          show_progress_bar=True
         ) 

    x_encoder.save("x-final-0313")

100%|█████████████████████████████████████████████████████████████████████| 2067765/2067765 [02:08<00:00, 16044.59it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17232 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████| 2067765/2067765 [02:37<00:00, 13146.23it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17232 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████| 2067766/2067766 [02:47<00:00, 12380.60it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17232 [00:00<?, ?it/s]

In [4]:
x_encoder.save("x-final-0313")

In [ ]:
# 1 fold 


train_dataset = pd.read_csv('train_0311.csv')


# 1 sentence_pairs= dev_examples, labels =dev_labels, - need for DEV dataset
dev_examples=[]
dev_labels=[]
#dev_dataset = train_dataset.sample(frac=0.01).copy()
tenperc= len(train_dataset) //100
dev_dataset = train_dataset.head(tenperc).copy()
for idx, row in tqdm(dev_dataset.iterrows(), total=dev_dataset.shape[0], position=0, leave=True):
    dev_examples.append([row['topic'], row['content']])
    dev_labels.append(row['label'])
print (len(train_dataset)) # now just exclude DEV from train dataset
train_dataset = train_dataset[~train_dataset['topic'].isin(dev_dataset['topic'])]
print (len(train_dataset))   

# 2 InputExample(texts=[row['topic'], row['content']], label=row['label']) - need for TRAIN  dataset
train_examples = []
for idx, row in tqdm(train_dataset.iterrows(), total=train_dataset.shape[0], position=0, leave=True):
    train_examples.append(InputExample(texts=[row['topic'], row['content']], label=row['label']))
    #train_examples.append(InputExample(texts=[row['query'], row['pos'], row['neg']]))
train_dataloader = DataLoader(train_examples, shuffle=False, batch_size=20)


#train_loss = losses.MultipleNegativesRankingLoss(model=x_encoder)

# 3 {‘query’: ‘’, ‘positive’: [], ‘negative’: []}  - need this for CERerankingEvaluator
#pos_dataset = pd.read_csv('train_0220_pos.csv')
#neg_dataset = pd.read_csv('train_0220_neg.csv')
#topics_data = pd.read_csv(os.path.join(BASE_DIR,'topics.csv')).query('has_content == True').sample(frac=0.03).copy()
#topics_data['merged_text'] = '[CLS]'+ topics_data['title'].fillna('')+'[SEP]'+topics_data['description'].fillna('')
#xencoder_eval_list = []
#for idx, row in tqdm(topics_data.iterrows(), total=topics_data.shape[0], position=0, leave=True):
#    curr_query = row['merged_text'] 
#    content_pos = pos_dataset[pos_dataset['topic_id_plus'] == row ['id']]['content_pos'].values.tolist()
#    content_neg = neg_dataset[neg_dataset['topic_id_plus'] == row ['id']]['content_neg'].values.tolist()
#    xencoder_eval_list.append ({'query':curr_query,'positive': content_pos, 'negative': content_neg})

#https://github.com/UKPLab/sentence-transformers/tree/master/examples/training/cross-encoder

num_epochs = 10  # 
model_save_path = "Xencoder-traininig"
 
from sentence_transformers.cross_encoder.evaluation import CERerankingEvaluator,CEBinaryAccuracyEvaluator
dev_evaluator = CEBinaryAccuracyEvaluator(sentence_pairs= dev_examples, labels =dev_labels, name='train-eval')
#dev_evaluator = CERerankingEvaluator(xencoder_eval_list, name='train-eval',write_csv=True)  

x_encoder.fit(train_dataloader=train_dataloader,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=dev_evaluator,
          evaluation_steps=30000,
          output_path=model_save_path,
          show_progress_bar=True
         ) 

x_encoder.save("Xencoder-0313")

# 4 re-generate examples

In [1]:
import sys
sys.path.append('/kaggle/input/sentence-transformers')
sys.path.append('/kaggle/input/sentence-embedding-models')
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
import string
from sentence_transformers import SentenceTransformer,InputExample,util, CrossEncoder,losses
import torch
from torch.utils.data import DataLoader
from sentence_transformers import losses

tqdm.pandas()

BASE_DIR = '/kaggle/input/learning-equality-curriculum-recommendations'
BASE_DIR = 'C:/learning'

content_data = pd.read_csv(os.path.join(BASE_DIR,'content.csv'))
correlations_data = pd.read_csv(os.path.join(BASE_DIR,'correlations.csv'))
topics_data = pd.read_csv(os.path.join(BASE_DIR,'topics.csv'))
sub_data = pd.read_csv(os.path.join(BASE_DIR,'sample_submission_0223.csv'))


#model_trained = SentenceTransformer('all-MiniLM-L6-v2-biencoder-trained-final-0304')
model_trained = SentenceTransformer('bi-final-0313b')
model_trained.max_seq_length = 128

# We will create some dictionaries and lists to move between topics, contents and targets
t2i = {k:i for i,k in enumerate(topics_data['id'].values.tolist())}
i2c = {i:k for i,k in enumerate(content_data['id'].values.tolist())}
topics_test = correlations_data['topic_id'].values.tolist()
targets_test = correlations_data['content_ids'].values.tolist()


#topics_data['merged_text'] = topics_data['title'].fillna(' ') + ' [SEP] ' + topics_data['description'].fillna(' ')
#content_data['merged_text'] =  content_data['title'].fillna(' ') + ' [SEP] ' + content_data['description'].fillna(' ') 


parentmap = topics_data.set_index('id')['title'].squeeze()
topics_data['parent_title'] = topics_data['parent'].map(parentmap)
topics_data['merged_text'] = topics_data['title'].fillna(' ').str.lower() + ' [SEP] ' + topics_data['description'].fillna(' ').str.lower() \
    +  ' [SEP] ' + topics_data['parent_title'].fillna(' ').str.lower()
content_data['merged_text'] =  content_data['title'].fillna(' ').str.lower() + ' [SEP] ' + content_data['description'].fillna(' ').str.lower()\
    +  ' [SEP] ' + content_data['text'].fillna(' ').str.lower().str[:100]


topic_emb = model_trained.encode(topics_data['merged_text'],  show_progress_bar = True) 
content_emb = model_trained.encode(content_data['merged_text'],  show_progress_bar = True)  # 2 минуты


import faiss
k = 50  # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

index = faiss.IndexFlatL2(content_emb.shape[-1])   # build the index
index.add(content_emb) # add vectors to the index
D, I = index.search(topic_emb, k) # actual search

topic_knn = [' '.join([i2c[x] for x in p]) for p in I.tolist()]
predictions = [topic_knn[t2i[i]] for i in topics_test]



t_list=[]
c_list=[]
l_list=[]

for tidx, j in tqdm(enumerate(predictions), total=len(predictions), position=0, leave=True):
    topic_id= correlations_data.iloc[tidx]['topic_id']
    query = topics_data.loc[topics_data['id']==topic_id]['merged_text'].fillna(' ').values[0]
    
    a = predictions[tidx].split()
    b = targets_test[tidx].split()
    a = [x for x in a if x not in b]    # remove all correct answers from a - keep mistakes only
    
 
    mask = content_data['id'].isin(a)
    neg_list = content_data.loc[mask]['merged_text'].to_list()
    for i in neg_list:
        t_list.append(query)
        c_list.append(i)
        l_list.append(0)        
   
    
    mask = content_data['id'].isin(b)
    pos_list = content_data.loc[mask]['merged_text'].to_list()
    for i in pos_list:
        t_list.append(query)
        c_list.append(i)
        l_list.append(1)
        

train_dataset = pd.DataFrame(list(zip(t_list, c_list,l_list)), columns = ['topic', 'content','label'])
train_dataset.to_csv('train_0311.csv', index = False)
print ('done')


Batches:   0%|          | 0/2406 [00:00<?, ?it/s]

Batches:   0%|          | 0/4814 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████| 61517/61517 [22:49<00:00, 44.91it/s]


done
